In [24]:
import import_hack
import numpy as np
import core.steps as steps
from core import data_store as ds
from core import transformer as tr
from core.common.ds_utils import print_ds_items_info
from os.path import join as pjoin
from core.search.exhaustive_searcher import ExhaustiveSearcher
from core.quantization.pq_quantizer import PQQuantizer, restore_from_clusters, build_pq_params_str
from core.metric.symmetric_distance_computer import SymmetricDistanceComputer
from core.common.path_helper import DataStoreHelper
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# SDC exhaustive search

In [25]:
ds_helper=DataStoreHelper(r'C:\data\computation\brodatz')

In [26]:
def sdc_search(descriptor_name, pq_params):
    centroids_ds=ds_helper.centroids_ds(descriptor_name, pq_params)
    centroids = ds.get_as_array(centroids_ds)
    pq_quantizer = restore_from_clusters(centroids)
    
    pqcodes_ds = ds_helper.pqcodes_ds(descriptor_name, pq_params)
    ids, pqcodes = ds.get_as_array(pqcodes_ds, return_ids=True)
  
    centroids_pairwise_distances_ds=ds_helper.centroids_pairwise_distances_ds(descriptor_name, pq_params)
    centroids_pairwise_distances = ds.get_as_array(centroids_pairwise_distances_ds)
    
    metric = SymmetricDistanceComputer(pq_quantizer, centroids_pairwise_distances)
    
    searcher_ = ExhaustiveSearcher(pqcodes, ids, metric=metric)
    n_nearest = 25
    
    neighbors_ids_ds=ds_helper.pq_search_neighbors_ids_ds('sdc', descriptor_name, pq_params)
    query_descriptors_ds=ds_helper.global_descriptors_ds(descriptor_name)
    
    steps.search_step(query_descriptors_ds, searcher_, n_nearest, neighbors_ids_ds)

In [27]:
descriptor_names=['histograms', 'lbphistograms', 'glcms']
choosen_bovwproductbincounts_pq_params= [{'n_clusters': K, 'n_quantizers': m} for K,m in [(64,4),(128,1),(16,1)]]
descriptor_names+=ds_helper.bovw_descriptors_names('bovwproductbincounts',choosen_bovwproductbincounts_pq_params)

In [28]:
K_arr = [2 ** i for i in [4,5,6,7,8]]
m_arr = [m for m in [1, 2, 4, 8, 16]]
pq_params_arr = [{'n_clusters': K, 'n_quantizers': m} for K in K_arr for m in m_arr]

In [29]:
for descriptor_name in descriptor_names:
    for pq_params in pq_params_arr:
        sdc_search(descriptor_name, pq_params)